## MLflow Tracking 퀵스타트
- 출처 : https://mlflow.org/docs/latest/getting-started/intro-quickstart/notebooks/index.html
- 수정사항 : 설명문을 한글로 번역

이 노트북에서는 로컬 MLflow 트래킹 서버를 사용하여 모델을 로깅, 등록한 다음   
일반 파이썬 함수(pyfunc)로 로드하여 Pandas 데이터 프레임에서 추론을 수행하는 방법을 보여줍니다.   
이 노트북에서는 MLflow 플루언트 API를 사용하여 MLflow 트래킹 서버와의 모든 상호 작용을 수행합니다.

In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

### MLflow Tracking URI 설정

이 노트북을 실행하는 위치에 따라 MLflow Tracking 서버와의 인터페이스 초기화 방법에 대한 구성이 다를 수 있습니다.   
    
이 예에서는 로컬에서 실행 중인 Tracking 서버를 사용하고 있지만 다른 옵션도 사용할 수 있습니다.   
가장 쉬운 방법은 [데이터브릭스 커뮤니티 에디션](https://community.cloud.databricks.com/) 내의 무료 관리형 서비스를 사용하는 것입니다.    
   

Tracking 서버 URL 설정 및 관리형 또는 자체 관리형 MLflow Tracking 서버에 대한 액세스 구성에 대한 자세한 내용은 [여기 노트북 실행 가이드](https://www.mlflow.org/docs/latest/getting-started/running-notebooks/index.html)를 참조하세요.

In [2]:
# NOTE: 위에서 언급한 링크를 검토하여 무료 데이터브릭스 커뮤니티 에디션과 같은 관리형 Tracking 서버에 연결하는 방법에 대한 지침을 확인하세요.

mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")

## 학습 데이터 로드 및 간단한 모델 학습하기

빠른 시작을 위해, scikit-learn에 포함되어 있는 익숙한 iris 데이터 세트를 사용하겠습니다.   
데이터를 분할한 후, 훈련 데이터에 대해 간단한 로지스틱 회귀 분류기를 훈련하고 
홀드아웃 테스트 데이터에서 몇 가지 오류 메트릭을 계산해 보겠습니다. 

이 부분의 유일한 MLflow 관련 활동은 모델의 하이퍼파라미터를 제공하기 위해 `param` dictionary룰 사용하는 것입니다.   
이는 모델과 관련 메타데이터를 로깅할 준비가 되었을 때 이러한 설정을 더 쉽게 로깅하기 위한 것입니다.

In [3]:
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

In [4]:
# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the model hyperparameters
params = {"solver": "lbfgs", "max_iter": 1000, "multi_class": "auto", "random_state": 8888}

# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Predict on the test set
y_pred = lr.predict(X_test)

# Calculate accuracy as a target loss metric
accuracy = accuracy_score(y_test, y_pred)

## MLflow 실험(Experiment) 정의하기

특정 프로젝트나 아이디어의 별개의 실행을 함께 그룹화하기 위해 각 반복(runs)을 함께 그룹화하는 실험을 정의할 수 있습니다.    
작업 중인 내용과 관련된 고유한 이름을 정의하면 정리하는 데 도움이 되고 나중에 실행을 찾기 위한 작업(검색)의 양을 줄일 수 있습니다. 

In [5]:
mlflow.set_experiment("MLflow Quickstart")

<Experiment: artifact_location='mlflow-artifacts:/333699500355693052', creation_time=1710560723576, experiment_id='333699500355693052', last_update_time=1710560723576, lifecycle_stage='active', name='MLflow Quickstart', tags={}>

## 모델, 하이퍼파라미터, 손실 메트릭을 MLflow에 기록합니다.

모델과 모델 훈련시 사용된 하이퍼파라미터, 홀드아웃 데이터에 대한 적합 모델 검증과 관련된 메트릭을 기록하기 위해 아래 그림과 같이 실행 컨텍스트를 시작합니다.   
이 컨텍스트의 범위 내에서, 호출하는 모든 fluent API(예: `mlflow.log_params()` 또는 `mlflow.sklearn.log_model()`)는 동일한 실행에 연결되고 함께 기록됩니다. 
fluent API :  사용자가 API를 자연스럽게 이해하고 사용할 수 있도록 설계된 API를 가리킵니다.   

In [6]:
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/03/19 23:24:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 4
Created version '4' of model 'tracking-quickstart'.


`mlflow.sklearn.load_model()` 을 사용하여 기본 scikit-learn 형식으로 모델을 다시 로드할 수 있지만,   
아래에서는 온라인 모델 서비스를 위해 이 모델을 로드하는 방식인 일반 Python 함수로 모델을 로드하고 있습니다.   
하지만 아래와 같이 batch 유스케이스에는 여전히 `pyfunc` 표현을 사용할 수 있습니다.

In [7]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

## 모델을 사용하여 Pandas DataFrame에서 iris 품종 예측하기

In [8]:
predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

# Convert X_test validation feature data to a Pandas DataFrame
result = pd.DataFrame(X_test, columns=iris_feature_names)

# Add the actual classes to the DataFrame
result["actual_class"] = y_test

# Add the model predictions to the DataFrame
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1
